In [34]:
import numpy as np
import cv2
import time

In [52]:
driveName = r"\\.\PhysicalDrive1"  # Change this to the correct drive

# SD Card sector information
headerSector =          1022 # Holds user settings to configure Miniscope and recording
configSector =          1023 # Holds final settings of the actual recording
dataStartSector =       1024 # Recording data starts here
sectorSize =            512

WRITE_KEY0 =				0x0D7CBA17
WRITE_KEY1 =				0x0D7CBA17
WRITE_KEY2 =				0x0D7CBA17
WRITE_KEY3 =				0x0D7CBA17

# SD Card Header Sector positions
HEADER_GAIN_POS =				4
HEADER_LED_POS =				5
HEADER_EWL_POS =				6
HEADER_RECORD_LENGTH_POS =  	7
HEADER_FRAME_RATE = 			8

# SD Card Config Sector positions
CONFIG_BLOCK_WIDTH_POS =			    0
CONFIG_BLOCK_HEIGHT_POS	=   		    1
CONFIG_BLOCK_FRAME_RATE_POS	=   	    2
CONFIG_BLOCK_BUFFER_SIZE_POS =  	    3
CONFIG_BLOCK_NUM_BUFFERS_RECORDED_POS =	4
CONFIG_BLOCK_NUM_BUFFERS_DROPPED_POS =	5

# Data Buffer Header positions
BUFFER_HEADER_HEADER_LENGTH_POS =			0
BUFFER_HEADER_LINKED_LIST_POS = 			1
BUFFER_HEADER_FRAME_NUM_POS	=   			2
BUFFER_HEADER_BUFFER_COUNT_POS =			3
BUFFER_HEADER_FRAME_BUFFER_COUNT_POS =  	4
BUFFER_HEADER_WRITE_BUFFER_COUNT_POS =  	5
BUFFER_HEADER_DROPPED_BUFFER_COUNT_POS =	6
BUFFER_HEADER_TIMESTAMP_POS	=   			7
BUFFER_HEADER_DATA_LENGTH_POS = 			8
BUFFER_HEADER_WRITE_TIMESTAMP_POS = 		9

In [62]:
# Needs to be run as administrator to have access to openning and reading card

# variables that you can adjust
correctDrive = False
f = open(driveName, "rb")  # Open drive

# Make sure this is the correct drive
# Read SD Card header and config sectors
f.seek(headerSector * sectorSize, 0)  # Move to correct sector
headerSectorData = np.frombuffer(f.read(sectorSize), dtype=np.uint32)
if ((WRITE_KEY0 == headerSectorData[0]) and (WRITE_KEY1 == headerSectorData[1]) and (WRITE_KEY2 == headerSectorData[2]) and (WRITE_KEY3 == headerSectorData[3])):
    correctDrive = True
    print("SD Card Openned.")
else:
    print ("Wrong Drive.")
    correctDrive = False
    f.close()

SD Card Openned.


In [4]:
    # Do not normally need to run this cell
    
    f.close()

In [63]:
# Load up Config Sector

f.seek(configSector * sectorSize, 0)  # Move to correct sector
configSectorData = np.fromstring(f.read(sectorSize), dtype=np.uint32)



# This needs to be removed once WF Miniscope writes this sector correctly
# configSectorData[CONFIG_BLOCK_WIDTH_POS] = 608
# configSectorData[CONFIG_BLOCK_HEIGHT_POS] = 608

configSectorData

C:\Users\dbaha\AppData\Local\Temp/ipykernel_10592/552914814.py:4: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  configSectorData = np.fromstring(f.read(sectorSize), dtype=np.uint32)


array([  304,   304,     0, 40960,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [86]:
# Read Data Sectors
frameNum = 0
pixelCount = 0

out = cv2.VideoWriter('WFV4_2.avi', cv2.VideoWriter_fourcc(*'GREY'), 
                      10.0, (configSectorData[CONFIG_BLOCK_WIDTH_POS], configSectorData[CONFIG_BLOCK_HEIGHT_POS] ), 
                      isColor=False)

frame = np.zeros((configSectorData[CONFIG_BLOCK_WIDTH_POS] * configSectorData[CONFIG_BLOCK_HEIGHT_POS], 1), dtype=np.uint8)
f.seek(dataStartSector * sectorSize, 0) # Starting data location
for i in range(1000):
    dataHeader = np.fromstring(f.read(4), dtype=np.uint32) # gets header length
    dataHeader = np.append(dataHeader, np.fromstring(f.read((dataHeader[BUFFER_HEADER_HEADER_LENGTH_POS] - 1) * 4), dtype=np.uint32))

    print(dataHeader)
    
    numBlocks = int((dataHeader[BUFFER_HEADER_DATA_LENGTH_POS] + (dataHeader[BUFFER_HEADER_HEADER_LENGTH_POS] * 4) + (512 - 1)) / 512)
    
    data = np.fromstring(f.read(numBlocks*512 - dataHeader[BUFFER_HEADER_HEADER_LENGTH_POS] * 4), dtype=np.uint8)

    # -------------------------------------
    if (dataHeader[BUFFER_HEADER_FRAME_BUFFER_COUNT_POS] == 0):
        # First buffer of a frame
        cv2.imshow('Video', np.reshape(frame, (configSectorData[CONFIG_BLOCK_WIDTH_POS], configSectorData[CONFIG_BLOCK_HEIGHT_POS] )))
        
        out.write(np.reshape(frame, (configSectorData[CONFIG_BLOCK_WIDTH_POS], configSectorData[CONFIG_BLOCK_HEIGHT_POS] )))
        cv2.imwrite('filename.jpeg', np.reshape(frame, (configSectorData[CONFIG_BLOCK_WIDTH_POS], configSectorData[CONFIG_BLOCK_HEIGHT_POS] )))
        
        cv2.waitKey(20)
            
        frame[0:dataHeader[BUFFER_HEADER_DATA_LENGTH_POS], 0] = data
        pixelCount = dataHeader[BUFFER_HEADER_DATA_LENGTH_POS]
        frameNum = dataHeader[BUFFER_HEADER_FRAME_NUM_POS]
    else:
        # All other buffers of a frame
        # startIdx = dataHeader[BUFFER_HEADER_FRAME_BUFFER_COUNT_POS] * 50 * 512
        frame[pixelCount:(pixelCount + dataHeader[BUFFER_HEADER_DATA_LENGTH_POS]), 0] = data[:dataHeader[BUFFER_HEADER_DATA_LENGTH_POS]]
        pixelCount = pixelCount + dataHeader[BUFFER_HEADER_DATA_LENGTH_POS]

            
out.release()
cv2.destroyWindow('Video')

C:\Users\dbaha\AppData\Local\Temp/ipykernel_10592/3443932903.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dataHeader = np.fromstring(f.read(4), dtype=np.uint32) # gets header length
C:\Users\dbaha\AppData\Local\Temp/ipykernel_10592/3443932903.py:13: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dataHeader = np.append(dataHeader, np.fromstring(f.read((dataHeader[BUFFER_HEADER_HEADER_LENGTH_POS] - 1) * 4), dtype=np.uint32))
C:\Users\dbaha\AppData\Local\Temp/ipykernel_10592/3443932903.py:19: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(f.read(numBlocks*512 - dataHeader[BUFFER_HEADER_HEADER_LENGTH_POS] * 4), dtype=np.uint8)


[   10     0     0     0     0     0     0   135 40920   135]
[   10     1     0     1     1     1     0   138 40920   179]
[   10     2     0     2     2     2     0   139 10568   222]
[   10     3     1     3     0     3     0   247 40920   247]
[   10     0     1     4     1     4     0   250 40920   290]
[   10     1     1     5     2     5     0   251 10568   333]
[   10     2     2     6     0     6     0   358 40920   358]
[   10     3     2     7     1     7     0   362 40920   402]
[   10     0     2     8     2     8     0   363 10568   445]
[   10     1     3     9     0     9     0   470 40920   470]
[   10     2     3    10     1    10     0   473 40920   513]
[   10     3     3    11     2    11     0   474 10568   556]
[   10     0     4    12     0    12     0   582 40920   582]
[   10     1     4    13     1    13     0   585 40920   625]
[   10     2     4    14     2    14     0   586 10568   668]
[   10     3     5    15     0    15     0   693 40920   694]
[   10  

OSError: [Errno 22] Invalid argument

In [78]:
cv2.VideoWriter_fourcc(*'MJPG')

1196444237